In [1]:
import pandas as pd
import numpy as np

In [2]:
def preprocess_data(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Exclude Cancel Invoices
    df = df[~df["InvoiceNo"].str.contains("C", na=False)]
    
    # Exclude Negative Quantities
    df = df[df["Quantity"] > 0]
    
    # Drop NA records
    df.dropna(inplace=True)
    
    # Add "TotalPrice" Column in Dataset
    df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]
    
    # Convert Datatype
    df["CustomerID"] = df["CustomerID"].astype(str)
    
    return df


file_path = r'C:\Users\mussie\Music\customer life time value\OnlineRetail.csv'
df = preprocess_data(file_path)
print(df.shape)


(397924, 9)


In [3]:
#Calculate TotalPrice Per Customer and Product
df = df.groupby(["CustomerID","StockCode"]).agg({"TotalPrice": "sum"}).sort_values("TotalPrice", ascending=False)

df.reset_index(inplace=True)

df.head()

,CustomerID,StockCode,TotalPrice
0,16446.0,23843,168469.60
1,12346.0,23166,77183.60
2,15098.0,22502,39916.50
3,18102.0,21623,16592.08
4,16210.0,21137,16027.92


In [4]:
#Calculate TotalPrice Per Product
df_product = df.groupby("StockCode").agg({"TotalPrice": "sum"}).sort_values("TotalPrice", ascending=False)

df_product.reset_index(inplace=True)

df_product.head()

,StockCode,TotalPrice
0,23843,168469.60
1,22423,142592.95
2,85123A,100603.50
3,85099B,85220.78
4,23166,81416.73


In [5]:
#Calculate Cumulative TotalPrice for Products
df_product['SumTotalPrice'] = df_product.TotalPrice.cumsum()

df_product.head()

,StockCode,TotalPrice,SumTotalPrice
0,23843,168469.60,168469.60
1,22423,142592.95,311062.55
2,85123A,100603.50,411666.05
3,85099B,85220.78,496886.83
4,23166,81416.73,578303.56


In [6]:
#Describe a Threshold
threshold = df_product["TotalPrice"].sum() * 0.80

threshold

7129126.3231999995

In [7]:
#Find products that make up 80% revenue.
df_ProductPer80 = df_product[df_product['SumTotalPrice'] <= threshold]

df_ProductPer80.head()

,StockCode,TotalPrice,SumTotalPrice
0,23843,168469.60,168469.60
1,22423,142592.95,311062.55
2,85123A,100603.50,411666.05
3,85099B,85220.78,496886.83
4,23166,81416.73,578303.56


In [8]:
df_ProductPer80["StockCode"].nunique()

776

In [9]:
#Calculate TotalPrice Per Product
df_customer = df.groupby("CustomerID").agg({"TotalPrice": "sum"}).sort_values("TotalPrice", ascending=False)

df_customer.reset_index(inplace=True)

df_customer.head()

,CustomerID,TotalPrice
0,14646.0,280206.02
1,18102.0,259657.30
2,17450.0,194550.79
3,16446.0,168472.50
4,14911.0,143825.06


In [13]:
#Calculate Cumulative TotalPrice for Products
df_customer['SumTotalPrice'] = df_customer.TotalPrice.cumsum()

df_customer

,CustomerID,TotalPrice,SumTotalPrice
0,14646.0,280206.02,280206.020
1,18102.0,259657.30,539863.320
2,17450.0,194550.79,734414.110
3,16446.0,168472.50,902886.610
4,14911.0,143825.06,1046711.670
...,...,...,...
4334,17956.0,12.75,8911391.054
4335,16454.0,6.90,8911397.954
4336,14792.0,6.20,8911404.154
4337,16738.0,3.75,8911407.904


In [14]:
#Describe a Threshold
threshold = df_customer["TotalPrice"].sum() * 0.80

threshold

7129126.3231999995

In [15]:
#Find customers that make up 80% revenue.
df_CustomerPer80 = df_customer[df_customer['SumTotalPrice'] <= threshold]

df_CustomerPer80

,CustomerID,TotalPrice,SumTotalPrice
0,14646.0,280206.02,280206.020
1,18102.0,259657.30,539863.320
2,17450.0,194550.79,734414.110
3,16446.0,168472.50,902886.610
4,14911.0,143825.06,1046711.670
...,...,...,...
1127,15660.0,1600.86,7121626.341
1128,16496.0,1599.52,7123225.861
1129,13850.0,1595.48,7124821.341
1130,17837.0,1593.38,7126414.721


In [16]:
df_CustomerPer80["CustomerID"].nunique()

1132